# Module 2: Frontier Models
## Lesson 10: The Chat Completions API (Under the Hood)

### 📄 Overview
Before relying on the convenient Python client libraries, this lesson breaks down the fundamental protocol of modern AI: the **Chat Completions API**. We perform a "Raw" HTTP request to understand exactly what data is being sent over the wire.

### 🗝️ Key Concepts
* **Chat Completions:** The industry-standard format for LLM interaction. You provide a list of message objects (history), and the model "completes" the conversation by predicting the next message.
* **The Endpoint:** A specific URL (e.g., `https://api.openai.com/v1/chat/completions`) that accepts JSON payloads and returns JSON predictions.
* **Client Libraries vs. Raw HTTP:**
    * **Client Library (`openai`)**: A Python wrapper that handles connection pooling, error retries, and converts JSON to Python objects.
    * **Raw HTTP**: The fundamental layer. Useful for debugging or when you are in a language without a supported library.

### 🛠️ Technical Implementation: The "Raw" Request
*Instead of `client.chat.completions.create`, we use the standard `requests` library to build the packet manually.*

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# 1. Define the Endpoint
url = "https://api.openai.com/v1/chat/completions"

# 2. Define the Headers (Authentication & Content Type)
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"  # The standard Bearer Token pattern
}

# 3. Define the Payload (The exact JSON schema OpenAI expects)
payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact about HTTP."}
    ]
}

print(f"📡 Sending POST request to {url}...")

# 4. Make the Request
response = requests.post(url, headers=headers, json=payload)

# 5. Parse the JSON Response
if response.status_code == 200:
    data = response.json()
    
    # Navigating the dictionary manually (The "Messy" part)
    content = data['choices'][0]['message']['content']
    
    print("\n✅ Success!")
    print(f"Response: {content}")
    print(f"\nFull JSON Structure:\n{json.dumps(data, indent=2)}")
else:
    print(f"❌ Error {response.status_code}: {response.text}")

### 🧪 Lab Notes & Engineering Log

*The following experiments focus on API Mechanics.*

#### Experiment 1: Why use the Library?
**Comparison:**
* **Raw HTTP (Above):** Requires manual header management, manual JSON parsing (`data['choices'][0]...`), and no built-in error handling.
* **Python Library:** Handles retries, types, and offers dot-notation (`response.choices[0]`).
* **Conclusion:** Use Raw HTTP only for debugging or extremely lightweight scripts where installing the full `openai` package is overkill.

#### Experiment 2: Inspecting the "Usage" Field
**Observation:**
In the raw JSON output above, I can clearly see the `usage` dictionary.
```json
"usage": {
  "prompt_tokens": 14,
  "completion_tokens": 12,
  "total_tokens": 26
}